In [ ]:
import itertools
import pprint
import sys

In [ ]:
sys.path.append("../src")
import hw_loading
import processor_utils
from processor_utils.units import FuncUnit, LockInfo, UnitModel
import program_utils
import sim_services
from str_utils import ICaseString

In [ ]:
capabilities = [ICaseString(cap) for cap in ["ALU", "MEM"]]
fetch_unit = UnitModel(ICaseString('F'), 1, capabilities, LockInfo(False, False), True)
decode_unit = UnitModel(ICaseString('D'), 1, capabilities, LockInfo(True, False), False)
execute_unit = UnitModel(ICaseString('X'), 1, capabilities, LockInfo(False, False), False)
memory_unit = UnitModel(ICaseString('M'), 1, capabilities, LockInfo(False, False), True)
writeback_unit = UnitModel(ICaseString('W'), 1, capabilities, LockInfo(False, True), False)

In [ ]:
proc_desc = processor_utils.ProcessorDesc([fetch_unit], [FuncUnit(writeback_unit, [memory_unit])], [], itertools.starmap(FuncUnit, [[memory_unit, [execute_unit]], [execute_unit, [decode_unit]], [decode_unit, [fetch_unit]]]))
hw_desc = hw_loading.HwDesc(proc_desc, processor_utils.load_isa([("LW", "MEM"), ("ADD", "ALU")], processor_utils.get_abilities(proc_desc)))

In [ ]:
prog = program_utils.compile_program(program_utils.read_program(["LW R1, (R2)", "ADD R3, R4, R5", "ADD R6, R7, R8", "ADD R9, R10, R11"]), hw_desc.isa)

In [ ]:
pprint.pp(sim_services.simulate(prog, sim_services.HwSpec(hw_desc.processor)))